In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [3]:
landsat = pd.read_csv(r'LANDSAT8_data_updated.csv')
landsat['date'] = pd.to_datetime(landsat['date'])
landsat.drop(columns=['lat', 'lon'], inplace=True)

In [4]:
modis_1 = pd.read_csv(r'MODIS_MOD09GA_data.csv')
modis_1['date'] = pd.to_datetime(modis_1['date'])
modis_1.drop(columns=['lat', 'lon'], inplace=True)

In [5]:
modis_2 = pd.read_csv(r'MODIS_MOD11A1_data.csv')
modis_2['date'] = pd.to_datetime(modis_2['date'])
modis_2.drop(columns=['lat', 'lon'], inplace=True)

In [6]:
modis_3 = pd.read_csv(r'MODIS_MOD16A2_data.csv')
modis_3['date'] = pd.to_datetime(modis_3['date'])
modis_3.drop(columns=['lat', 'lon'], inplace=True)

In [7]:
modis_4 = pd.read_csv(r'MODIS_MOD13Q1_data.csv')
modis_4['date'] = pd.to_datetime(modis_4['date'])
modis_4.drop(columns=['lat', 'lon'], inplace=True)

In [8]:
modis_5 = pd.read_csv(r'MODIS_MCD43A4_data.csv')
modis_5['date'] = pd.to_datetime(modis_5['date'])
modis_5.drop(columns=['lat', 'lon'], inplace=True)

In [134]:
sentinel_1 = pd.read_csv(r'Sentinel1_data.csv')
sentinel_1['date'] = pd.to_datetime(sentinel_1['date'])
sentinel_1.drop(columns=['lat', 'lon'], inplace=True)

In [170]:
sentinel_2 = pd.read_csv(r'Sentinel2_data.csv')
sentinel_2['date'] = pd.to_datetime(sentinel_2['date'])
sentinel_2.drop(columns=['lat', 'lon'], inplace=True)

In [11]:
satelites = [modis_1, modis_2, modis_3, modis_4, modis_5, sentinel_1, sentinel_2, landsat]
satelite_names = ['modis_1', 'modis_2', 'modis_3', 'modis_4', 'modis_5', 'sentinel_1', 'sentinel_2', 'landsat']

# landsat

In [12]:
print("landsat Columns:\n", landsat.columns, "\n")

landsat Columns:
 Index(['QA_PIXEL', 'QA_RADSAT', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5',
       'SR_B6', 'SR_B7', 'ST_B10', 'date', 'PID'],
      dtype='object') 



In [13]:
landsat.isnull().sum()

QA_PIXEL     0
QA_RADSAT    0
SR_B1        0
SR_B2        0
SR_B3        0
SR_B4        0
SR_B5        0
SR_B6        0
SR_B7        0
ST_B10       0
date         0
PID          0
dtype: int64

In [14]:
def is_good_pixel(qa_pixel):
    """
    Returns True if pixel is not affected by fill, cloud, shadow, or snow.
    """
    return (
        (qa_pixel & (1 << 0)) == 0 and  # Fill
        (qa_pixel & (1 << 1)) == 0 and  # Dilated cloud
        (qa_pixel & (1 << 3)) == 0 and  # Cloud
        (qa_pixel & (1 << 4)) == 0 and  # Cloud shadow
        (qa_pixel & (1 << 5)) == 0      # Snow
    )

def is_not_saturated(qa_radsat, band_bits):
    """
    Returns True if none of the specified band bits are saturated.
    band_bits is a list of bits corresponding to SR_Bx bands.
    """
    return all((qa_radsat & (1 << bit)) == 0 for bit in band_bits)

def clean_landsat(df):
    band_to_bit = {
        'SR_B1': 0, 'SR_B2': 1, 'SR_B3': 2,
        'SR_B4': 3, 'SR_B5': 4, 'SR_B6': 5, 'SR_B7': 6
    }

    def row_is_clean(row):
        used_bands = [b for b in band_to_bit if b in row.index]
        band_bits = [band_to_bit[b] for b in used_bands]
        return is_good_pixel(row['QA_PIXEL']) and is_not_saturated(row['QA_RADSAT'], band_bits)

 
    mask = df.apply(row_is_clean, axis=1)
    return df[mask].copy()

landsat = clean_landsat(landsat)


In [15]:
landsat.drop(columns=['QA_PIXEL','QA_RADSAT'],inplace=True)

In [16]:
print(landsat.duplicated().sum())

0


In [17]:
print(landsat.duplicated(subset=['PID', 'date']).sum())

1176699


### Multiple sensing per day
#### aggregating with median

In [18]:
landsat = landsat.groupby(['PID', 'date']).median().reset_index()

In [19]:
landsat.rename(columns={
    'SR_B1': 'b01',
   'SR_B2': 'b02',
   'SR_B3': 'b03',
    'SR_B4': 'b04',
    'SR_B5':'b05',
    'SR_B6':'b06',
    'SR_B7':'b07',
}, inplace=True)

In [21]:
landsat

,PID,date,b01,b02,b03,b04,b05,b06,b07,ST_B10
0,ID_002W8m,2017-01-22,0.031165,0.041725,0.064962,0.085587,0.178620,0.206643,0.152000,1.079025
1,ID_002W8m,2017-02-07,0.029075,0.041862,0.070545,0.099007,0.182690,0.223775,0.159013,1.094288
2,ID_002W8m,2017-02-23,0.031797,0.048517,0.081242,0.112262,0.194927,0.228038,0.168225,1.126545
3,ID_002W8m,2017-03-10,0.030450,0.042660,0.071232,0.104370,0.187695,0.225810,0.170590,1.145878
4,ID_002W8m,2017-03-26,0.030505,0.040955,0.065568,0.092957,0.165502,0.195450,0.145042,1.098798
...,...,...,...,...,...,...,...,...,...,...
454494,ID_zzVFDw,2019-09-10,0.022860,0.030340,0.075962,0.053852,0.369910,0.175705,0.088090,1.031753
454495,ID_zzVFDw,2019-09-26,0.026792,0.036115,0.082810,0.064220,0.381350,0.190390,0.100300,1.045200
454496,ID_zzVFDw,2019-10-12,0.033447,0.045162,0.085120,0.091830,0.344555,0.210602,0.115700,1.083700
454497,ID_zzVFDw,2019-10-28,0.025060,0.031192,0.066310,0.058775,0.289390,0.167592,0.092187,1.025152


In [24]:
landsat.to_csv('landsat_clean.csv', index=False)

# modis_1

In [33]:
print("MODIS MOD09GA Columns:\n", modis_1.columns, "\n")

MODIS MOD09GA Columns:
 Index(['date', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04',
       'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'PID'],
      dtype='object') 



In [37]:
modis_1.isnull().sum()

date            0
sur_refl_b01    0
sur_refl_b02    0
sur_refl_b03    0
sur_refl_b04    0
sur_refl_b05    0
sur_refl_b06    0
sur_refl_b07    0
PID             0
dtype: int64

In [22]:
print(landsat.duplicated().sum())

0


In [23]:
print(landsat.duplicated(subset=['PID', 'date']).sum())

0


In [25]:
modis_1.rename(columns={'sur_refl_b01': 'b01',
                        'sur_refl_b02': 'b02',
                       'sur_refl_b03': 'b03',
                       'sur_refl_b04': 'b04',
                        'sur_refl_b05': 'b05',
                       'sur_refl_b06': 'b06',
                        'sur_refl_b07': 'b07'}, inplace=True)


In [29]:
modis_1= modis_1[['PID', 'date', 'b01', 'b02', 'b03', 'b04', 'b05', 'b06', 'b07']]


In [30]:
modis_1

,PID,date,b01,b02,b03,b04,b05,b06,b07
0,ID_I5RGjv,2017-01-01,0.1997,0.2713,0.1469,0.1705,0.3020,0.2922,0.1887
1,ID_8jWzJ5,2017-01-01,0.2831,0.3459,0.2686,0.2642,0.3604,0.3223,0.2089
2,ID_d009mj,2017-01-01,0.1619,0.2510,0.0697,0.1162,0.2792,0.2783,0.1895
3,ID_2ymCjP,2017-01-01,0.1520,0.2482,0.0619,0.1032,0.2814,0.2783,0.1878
4,ID_YIft2H,2017-01-01,0.1664,0.2496,0.0727,0.1187,0.2980,0.2710,0.1924
...,...,...,...,...,...,...,...,...,...
7486442,ID_hH2j07,2019-12-30,0.0808,0.2540,0.0312,0.0619,0.2863,0.2501,0.1461
7486443,ID_SL3ETR,2019-12-30,0.0985,0.2763,0.0412,0.0758,0.2880,0.2690,0.1533
7486444,ID_GGQ39P,2019-12-30,0.0826,0.2633,0.0342,0.0656,0.2882,0.2433,0.1391
7486445,ID_PS0d4J,2019-12-30,0.0595,0.2161,0.0283,0.0493,0.2635,0.1755,0.1045


In [31]:
modis_1.to_csv('modis_1_clean.csv', index=False)

# modis_2

In [32]:
print("MODIS MOD11A1 Columns:\n", modis_2.columns, "\n")

MODIS MOD11A1 Columns:
 Index(['LST_Day_1km', 'LST_Night_1km', 'date', 'PID'], dtype='object') 



In [33]:
modis_2.isnull().sum()

LST_Day_1km      0
LST_Night_1km    0
date             0
PID              0
dtype: int64

In [35]:
print(modis_2.duplicated().sum())

0


In [36]:
print(modis_2.duplicated(subset=['PID', 'date']).sum())

3872


### Multiple sensing per day
#### aggregating with median

In [37]:
modis_2 = modis_2.groupby(['PID', 'date']).median().reset_index()

In [39]:
modis_2.to_csv('modis_2_clean.csv', index=False)

In [40]:
modis_2

,PID,date,LST_Day_1km,LST_Night_1km
0,ID_002W8m,2017-01-01,299.12,289.12
1,ID_002W8m,2017-01-02,301.90,289.66
2,ID_002W8m,2017-01-08,302.84,288.92
3,ID_002W8m,2017-01-09,307.30,288.84
4,ID_002W8m,2017-01-10,304.08,288.94
...,...,...,...,...
2461914,ID_zzVFDw,2019-12-25,302.06,284.20
2461915,ID_zzVFDw,2019-12-26,299.42,284.34
2461916,ID_zzVFDw,2019-12-27,302.42,286.16
2461917,ID_zzVFDw,2019-12-28,303.08,286.34


# modis_3

In [41]:
print("MODIS MOD16A2 Columns:\n", modis_3.columns, "\n")

MODIS MOD16A2 Columns:
 Index(['ET', 'PET', 'date', 'PID'], dtype='object') 



In [43]:
modis_3.isnull().sum()

ET      0
PET     0
date    0
PID     0
dtype: int64

In [44]:
print(modis_3.duplicated().sum())

0


In [45]:
print(modis_3.duplicated(subset=['PID', 'date']).sum())

0


In [47]:
modis_3=modis_3[['PID','date','ET','PET']]

In [49]:
modis_3.to_csv('modis_3_clean.csv', index=False)

In [48]:
modis_3

,PID,date,ET,PET
0,ID_UgzkN8,2017-01-01,0.9,59.9
1,ID_d009mj,2017-01-01,0.9,60.4
2,ID_2ymCjP,2017-01-01,1.3,59.6
3,ID_YIft2H,2017-01-01,0.9,59.4
4,ID_ZaKEFI,2017-01-01,1.0,58.9
...,...,...,...,...
935358,ID_hH2j07,2019-12-27,7.1,29.4
935359,ID_SL3ETR,2019-12-27,6.4,29.2
935360,ID_GGQ39P,2019-12-27,6.4,29.5
935361,ID_PS0d4J,2019-12-27,6.8,29.5


# modis_4

In [50]:
print("MODIS MOD13Q1 Columns:\n", modis_4.columns, "\n")

MODIS MOD13Q1 Columns:
 Index(['EVI', 'NDVI', 'RelativeAzimuth', 'SolarZenith', 'ViewZenith', 'date',
       'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07', 'PID'],
      dtype='object') 



In [55]:
modis_4.isnull().sum()

EVI                0
NDVI               0
RelativeAzimuth    0
SolarZenith        0
ViewZenith         0
date               0
sur_refl_b01       0
sur_refl_b02       0
sur_refl_b03       0
sur_refl_b07       0
PID                0
dtype: int64

In [56]:
modis_4['RelativeAzimuth'].describe()

count    545563.000000
mean          0.070720
std           1.017931
min          -1.800000
25%          -0.482300
50%           0.080700
75%           0.987700
max           1.799900
Name: RelativeAzimuth, dtype: float64

In [58]:
modis_4['SolarZenith'].describe()

count    545563.000000
mean          0.273643
std           0.068684
min           0.109700
25%           0.219700
50%           0.259300
75%           0.333400
max           0.447200
Name: SolarZenith, dtype: float64

In [59]:
modis_4['ViewZenith'].describe()

count    545563.000000
mean          0.195599
std           0.148687
min           0.000200
25%           0.075100
50%           0.148000
75%           0.290300
max           0.649000
Name: ViewZenith, dtype: float64

In [61]:
modis_4.drop(columns=['RelativeAzimuth','SolarZenith','ViewZenith'],inplace=True)

In [66]:
modis_4.rename(columns={'sur_refl_b01': 'b01',
                        'sur_refl_b02': 'b02',
                       'sur_refl_b03': 'b03',
                        'sur_refl_b07': 'b07'}, inplace=True)


In [69]:
modis_4=modis_4[['PID','date','b01','b02','b03','b07','EVI','NDVI']]

In [71]:
print(modis_4.duplicated().sum())

0


In [72]:
print(modis_4.duplicated(subset=['PID', 'date']).sum())

0


In [73]:
modis_4

,PID,date,b01,b02,b03,b07,EVI,NDVI
0,ID_I5RGjv,2017-01-01,0.2120,0.3041,0.0909,0.2675,0.1211,0.1779
1,ID_8jWzJ5,2017-01-01,0.1969,0.2905,0.0807,0.2514,0.1242,0.1897
2,ID_d009mj,2017-01-01,0.1562,0.2421,0.0738,0.2404,0.1329,0.2175
3,ID_2ymCjP,2017-01-01,0.1701,0.2635,0.0730,0.2624,0.1343,0.2155
4,ID_YIft2H,2017-01-01,0.2059,0.2843,0.0846,0.2816,0.1039,0.1600
...,...,...,...,...,...,...,...,...
545558,ID_hH2j07,2019-12-19,0.0845,0.2980,0.0379,0.1473,0.3510,0.5584
545559,ID_SL3ETR,2019-12-19,0.1138,0.3200,0.0420,0.1666,0.3055,0.4755
545560,ID_GGQ39P,2019-12-19,0.0961,0.3154,0.0363,0.1525,0.3385,0.5330
545561,ID_PS0d4J,2019-12-19,0.0641,0.2692,0.0304,0.1210,0.3596,0.6151


In [74]:
modis_4.to_csv('modis_4_clean.csv', index=False)

# modis_5

In [75]:
print("MODIS MCD43A4 Columns:\n", modis_5.columns, "\n")

MODIS MCD43A4 Columns:
 Index(['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2',
       'Nadir_Reflectance_Band3', 'Nadir_Reflectance_Band4', 'date', 'PID'],
      dtype='object') 



In [76]:
modis_5.isnull().sum()

Nadir_Reflectance_Band1    0
Nadir_Reflectance_Band2    0
Nadir_Reflectance_Band3    0
Nadir_Reflectance_Band4    0
date                       0
PID                        0
dtype: int64

In [77]:
modis_5.rename(columns={'Nadir_Reflectance_Band1': 'b01',
                        'Nadir_Reflectance_Band2': 'b02',
                       'Nadir_Reflectance_Band3': 'b03',
                        'Nadir_Reflectance_Band4': 'b04'}, inplace=True)

In [80]:
print(modis_5.duplicated().sum())

2


In [82]:
modis_5.drop_duplicates(keep='first', inplace=True)

In [81]:
print(modis_5.duplicated(subset=['PID', 'date']).sum())

7908


In [83]:
modis_5= modis_5.groupby(['PID', 'date']).median().reset_index()

In [86]:
modis_5.to_csv('modis_5_clean.csv', index=False)

In [87]:
modis_5

,PID,date,b01,b02,b03,b04
0,ID_002W8m,2017-01-01,0.0742,0.2430,0.0273,0.0590
1,ID_002W8m,2017-01-02,0.0762,0.2427,0.0275,0.0596
2,ID_002W8m,2017-01-03,0.0756,0.2430,0.0278,0.0593
3,ID_002W8m,2017-01-04,0.0758,0.2431,0.0280,0.0594
4,ID_002W8m,2017-01-05,0.0770,0.2421,0.0283,0.0600
...,...,...,...,...,...,...
7759468,ID_zzVFDw,2019-12-26,0.0727,0.2945,0.0367,0.0705
7759469,ID_zzVFDw,2019-12-27,0.0741,0.2888,0.0375,0.0710
7759470,ID_zzVFDw,2019-12-28,0.0743,0.2895,0.0376,0.0713
7759471,ID_zzVFDw,2019-12-29,0.0754,0.2882,0.0376,0.0722


# sentinel_1

In [135]:
print("Sentinel-1 Columns:\n", sentinel_1.columns, "\n")

Sentinel-1 Columns:
 Index(['VH', 'VV', 'date', 'instrumentMode', 'orbitProperties_pass',
       'relativeOrbitNumber_start', 'PID'],
      dtype='object') 



In [136]:
sentinel_1.isnull().sum()

VH                           0
VV                           0
date                         0
instrumentMode               0
orbitProperties_pass         0
relativeOrbitNumber_start    0
PID                          0
dtype: int64

In [137]:
sentinel_1.drop(columns=['instrumentMode'],inplace=True)

In [138]:
sentinel_1['relativeOrbitNumber_start'].describe()

count    2.175275e+06
mean     1.100202e+02
std      4.364981e+01
min      6.000000e+00
25%      7.900000e+01
50%      7.900000e+01
75%      1.520000e+02
max      1.600000e+02
Name: relativeOrbitNumber_start, dtype: float64

In [139]:
mapping = {'ASCENDING': 1, 'DESCENDING': 0}
sentinel_1['orbitProperties_pass_encoded'] = sentinel_1['orbitProperties_pass'].map(mapping)

In [140]:
sentinel_1.drop(columns=['orbitProperties_pass'],inplace=True)

In [141]:
sentinel_1=sentinel_1[['PID','date','VH','VV','relativeOrbitNumber_start','orbitProperties_pass_encoded']]

In [142]:
print(sentinel_1.duplicated().sum())

0


In [143]:
print(sentinel_1.duplicated(subset=['PID', 'date']).sum())

1471403


In [155]:
sentinel_1 = (
    sentinel_1.groupby(['PID', 'date'])[['VH','VV','relativeOrbitNumber_start']]
    .median()
    .reset_index()
)


In [157]:
sentinel_1.to_csv('sentinel_1_clean.csv', index=False)

In [156]:
sentinel_1

,PID,date,VH,VV,relativeOrbitNumber_start
0,ID_002W8m,2017-01-20,-22.818425,-16.511257,50.0
1,ID_002W8m,2017-04-08,-26.675698,-20.244435,152.0
2,ID_002W8m,2017-05-31,-18.184365,-15.039445,50.0
3,ID_002W8m,2017-07-13,-18.064261,-8.023376,152.0
4,ID_002W8m,2017-09-28,-18.613407,-9.625051,50.0
...,...,...,...,...,...
703867,ID_zzVFDw,2019-11-07,-14.869617,-9.049269,79.0
703868,ID_zzVFDw,2019-11-19,-15.019815,-5.202753,79.0
703869,ID_zzVFDw,2019-12-01,-14.605878,-6.827098,79.0
703870,ID_zzVFDw,2019-12-13,-16.319907,-6.621517,79.0


# sentinel_2

In [171]:
print("Sentinel-2 Columns:\n", sentinel_2.columns, "\n")

Sentinel-2 Columns:
 Index(['B1', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A',
       'B9', 'CLOUDY_PIXEL_PERCENTAGE', 'MEAN_SOLAR_ZENITH_ANGLE',
       'NODATA_PIXEL_PERCENTAGE', 'SENSING_ORBIT_NUMBER', 'SPACECRAFT_NAME',
       'date', 'PID'],
      dtype='object') 



In [172]:
sentinel_2.isnull().sum()

B1                         0
B11                        0
B12                        0
B2                         0
B3                         0
B4                         0
B5                         0
B6                         0
B7                         0
B8                         0
B8A                        0
B9                         0
CLOUDY_PIXEL_PERCENTAGE    0
MEAN_SOLAR_ZENITH_ANGLE    0
NODATA_PIXEL_PERCENTAGE    0
SENSING_ORBIT_NUMBER       0
SPACECRAFT_NAME            0
date                       0
PID                        0
dtype: int64

In [ ]:
# One-hot encode 'SENSING_ORBIT_NUMBER' and convert booleans to integers
orbit_dummies = pd.get_dummies(sentinel_2['SENSING_ORBIT_NUMBER'], prefix='orbit').astype(int)

# Concatenate the one-hot encoded columns back to the dataframe
sentinel_2 = pd.concat([sentinel_2, orbit_dummies], axis=1)

# Drop the original column if you want
sentinel_2 = sentinel_2.drop(columns=['SENSING_ORBIT_NUMBER','SPACECRAFT_NAME'])


In [189]:
# Using relaxed nodata threshold to keep more data
cloud_threshold = 20.0
nodata_threshold = 20.0
sentinel_2 = sentinel_2[
    (sentinel_2['CLOUDY_PIXEL_PERCENTAGE'] <= cloud_threshold) &
    (sentinel_2['NODATA_PIXEL_PERCENTAGE'] <= nodata_threshold)
].copy()


In [190]:

print(len(sentinel_2))

6764


In [195]:
sentinel_2.columns

Index(['B1', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A',
       'B9', 'date', 'PID', 'orbit_35.0', 'orbit_49.0', 'orbit_135.0'],
      dtype='object')

In [193]:
sentinel_2.drop(columns=['CLOUDY_PIXEL_PERCENTAGE','MEAN_SOLAR_ZENITH_ANGLE','NODATA_PIXEL_PERCENTAGE'],inplace=True)

In [196]:
sentinel_2=sentinel_2[['PID','date','B1', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A',
       'B9','orbit_35.0','orbit_49.0','orbit_135.0']]

In [200]:
sentinel_2.drop(columns=['orbit_35.0','orbit_49.0','orbit_135.0'],inplace=True)

In [202]:
print(sentinel_2.duplicated().sum())

0


In [203]:
print(sentinel_2.duplicated(subset=['PID', 'date']).sum())

4565


In [204]:
sentinel_2= sentinel_2.groupby(['PID', 'date']).median().reset_index()

In [208]:
sentinel_2.drop(columns=['B1','B9'],inplace=True)

In [209]:
sentinel_2

,PID,date,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A
0,ID_002W8m,2019-12-18,2195.5,1669.5,367.0,541.5,682.0,967.5,1344.0,1533.5,1531.5,1705.5
1,ID_00Mug5,2019-12-18,1526.0,974.5,445.0,657.0,783.0,1055.0,1470.0,1684.0,1923.0,1849.5
2,ID_03sGRz,2019-12-18,2139.0,1355.0,323.0,556.5,606.0,1168.0,1738.0,2035.5,1962.0,2305.5
3,ID_046OrE,2019-12-18,2325.0,1672.0,464.0,797.5,1147.0,1398.0,1735.0,1934.0,2046.0,2153.0
4,ID_04AslO,2019-12-18,1339.0,725.5,150.0,321.0,263.0,662.5,1669.5,2052.5,2069.0,2253.5
...,...,...,...,...,...,...,...,...,...,...,...,...
2194,ID_zuLVyR,2019-12-18,1758.5,1029.0,280.5,651.5,572.0,1215.5,2431.5,2788.5,2895.0,2944.0
2195,ID_zv8S8L,2019-06-21,3017.0,2709.0,1751.0,2068.0,2414.0,2611.0,2820.0,2930.0,3132.0,2979.0
2196,ID_zv8S8L,2019-10-04,2577.0,1948.0,1468.0,1913.0,1830.0,2537.0,3471.0,3639.0,3702.0,3797.0
2197,ID_zvd9BI,2019-12-18,868.0,519.0,204.0,348.0,324.5,635.0,1233.0,1346.0,1295.5,1483.0


In [210]:
sentinel_2.to_csv('sentinel_2_clean.csv', index=False)